<a href="https://colab.research.google.com/github/nathaniellamjohnson/CAISNews-Headlines-for-Sarcasm-Detection-Winter-Project/blob/main/CAISNews_Headlines_for_Sarcasm_Detection_Winter_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check we're using GPU
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [25]:
# Working code snippet for downloading kaggle data
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"nljnlj","key":"6276389842ffd79fb5a4843dba3bebc0"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [28]:
# If shows up, working

! kaggle datasets list

ref                                                             title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ahsan81/hotel-reservations-classification-dataset               Hotel Reservations Dataset                   480KB  2023-01-04 12:50:31           1531         60  1.0              
thedevastator/global-fossil-co2-emissions-by-country-2002-2022  Emissions by Country, 2002-2022              621KB  2023-01-02 20:10:36            893         35  1.0              
die9origephit/fifa-world-cup-2022-complete-dataset              Fifa World Cup 2022: Complete Dataset          7KB  2022-12-18 22:51:11           5000        164  1.0              
rakkesharv/spotify-top-10000-streamed-songs                     Spotify Top 10000 Streamed Song

In [29]:
! kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 216MB/s]


In [32]:
# Unpacks zip file using shutil into /data
import shutil 
shutil.unpack_archive("news-headlines-dataset-for-sarcasm-detection.zip", "/content/data")

In [41]:
! cd data
!pwd

/content


In [43]:
# Paths to Folders for Test & Train
! ls /content/data
path_1 = "/content/data/Sarcasm_Headlines_Dataset.json"
path_2 = "/content/data/Sarcasm_Headlines_Dataset_v2.json"
    # Folders for a-z, nothing, space, del

Sarcasm_Headlines_Dataset.json	Sarcasm_Headlines_Dataset_v2.json


In [49]:
# Read both jsons, convert to dataframes
import pandas as pd 

a = pd.read_json(path_1, lines=True)
b = pd.read_json(path_2, lines=True)
b.sample(10)

,is_sarcastic,headline,article_link
12240,0,7 super seeds with big health benefits,https://www.huffingtonpost.com/entry/7-super-s...
8922,1,kerry vows to raise wife's taxes,https://politics.theonion.com/kerry-vows-to-ra...
13066,1,bride and groom clearly have not kissed much,https://local.theonion.com/bride-and-groom-cle...
20911,0,hbo's martin luther king jr. film reveals his ...,https://www.huffingtonpost.com/entry/hbo-marti...
8031,1,documentary viewer can't wait to find out whic...,https://local.theonion.com/documentary-viewer-...
10128,1,defensive clinton campaign releases new 'who a...,https://politics.theonion.com/defensive-clinto...
23817,0,'the simpsons' duff beer will soon be a reality,https://www.huffingtonpost.com/entry/the-simps...
23402,0,phyllis schlafly and the kingmakers,https://www.huffingtonpost.com/entry/phyllis-s...
24335,1,hero publicist honored,https://entertainment.theonion.com/hero-public...
27242,1,single napkin accompanying takeout order presu...,https://local.theonion.com/single-napkin-accom...


In [51]:
# Create master dataframe by appending
df = a.append(b)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [52]:
# Get the lists of sentences and their labels.
headlines = df.headline.values
labels = df.is_sarcastic.values

In [57]:
# Load Bert Tokenizer
! pip install transformers
import transformers
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.7 MB/s eta 0:00:00


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]